In [2]:
!pip install apache-airflow==3.1.7

INFO: pip is looking at multiple versions of cadwyn to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of cadwyn to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.9/304.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 8.7 MB/s eta 0:00:00
 

In [3]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime
import subprocess

default_args = {
    "owner": "airflow",
    "retries": 1,
}

def preprocess():
    subprocess.run(["python", "scripts/preprocess.py"], check=True)

def train():
    subprocess.run(["python", "scripts/train_model.py"], check=True)

def evaluate():
    subprocess.run(["python", "scripts/evaluate.py"], check=True)

with DAG(
    dag_id="job_classification_pipeline",
    start_date=datetime(2024, 1, 1),
    schedule="@daily",
    catchup=False,
    default_args=default_args,
    description="Automated ML retraining pipeline for job classification",
    tags=["ml", "airflow", "training"]
) as dag:

    preprocess_task = PythonOperator(
        task_id="preprocess_data",
        python_callable=preprocess,
    )

    train_task = PythonOperator(
        task_id="train_model",
        python_callable=train,
    )

    evaluate_task = PythonOperator(
        task_id="evaluate_model",
        python_callable=evaluate,
    )

    preprocess_task >> train_task >> evaluate_task

/tmp/ipython-input-1795764867.py:2 DeprecatedImportWarning: The `airflow.operators.python.PythonOperator` attribute is deprecated. Please use `'airflow.providers.standard.operators.python.PythonOperator'`.